In [43]:
from Functions.documents import load_documents
from Functions.text import clear_text, remove_stopwords, split_text_into_chunks, remove_portuguese_accents



DOCUMENT_PATHS = ["./Documents/a (1).pdf", "./Documents/FAQ (1).pdf"]
CHUNK_SIZE = 500



# BEGIN = Processing documents 
texts = load_documents(DOCUMENT_PATHS)
cleaned_texts = [clear_text(text) for text in texts]
no_stopwords_text = [remove_stopwords(text) for text in cleaned_texts]
no_accents_portuguese = [remove_portuguese_accents(text) for text in no_stopwords_text]
joined_texts = " ".join(no_accents_portuguese)
joined_texts = joined_texts.lower()
text_chunks = split_text_into_chunks(joined_texts, chunk_size=CHUNK_SIZE)
# END = Processing documents 


In [ ]:
len(text_chunks)

In [ ]:
len(text_chunks[0])

In [ ]:
from sentence_transformers import SentenceTransformer, util


import gym
import numpy as np
from gym import spaces
from sentence_transformers import SentenceTransformer, util
from Functions.documents import load_documents
from Functions.text import clear_text, remove_stopwords, split_text_into_chunks

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [10]:
encoded = model.encode(text_chunks[0])

In [11]:
not_encoded = text_chunks[0]

In [ ]:
len(encoded)

In [ ]:
len(not_encoded)

In [14]:
document_embeddings = [model.encode(chunk) for chunk in text_chunks] 

In [22]:
question_embedding = model.encode("Como e quando posso realizar o trancamento de matrícula?")

In [ ]:
from sentence_transformers import util

# Compute cosine similarity between the question and all document chunks
cosine_similarities = [util.pytorch_cos_sim(question_embedding, doc_emb).item() for doc_emb in document_embeddings]

# Convert cosine similarity to a distance measure (1 - similarity)
cosine_distances = [1 - sim for sim in cosine_similarities]

print(cosine_distances)  # List of distances (lower means more similar)

In [ ]:
# Find the index of the most similar (highest similarity) and least similar (lowest similarity) document chunks
max_index = max(range(len(cosine_similarities)), key=lambda i: cosine_similarities[i])
min_index = min(range(len(cosine_similarities)), key=lambda i: cosine_similarities[i])

print(f"Most relevant document chunk index: {max_index} (Score: {cosine_similarities[max_index]})")
print(f"Least relevant document chunk index: {min_index} (Score: {cosine_similarities[min_index]})")


In [ ]:
print(text_chunks[2])

In [ ]:
import textdistance
# https://github.com/life4/textdistance?tab=readme-ov-file

f1 = "o carro está andando"
# f2 = "o ônibus está em movimento"
f2 = "o ônibus andando"
print("monge: ",textdistance.monge_elkan(f1,f2))
print("cosine: ",textdistance.cosine(f1,f2))
print("hamming: ",textdistance.hamming.normalized_similarity(f1,f2))

In [ ]:

# Compute cosine similarity between the question and all document chunks
cosine_similarities = [util.pytorch_cos_sim(self.query_embedding, doc_emb).item() for doc_emb in self.document_embeddings]

# Convert cosine similarity to a distance measure (1 - similarity)
cosine_distances = [1 - sim for sim in cosine_similarities]

self.similarities = np.array(cosine_distances.copy())

In [1]:
# Testando ambiente separadamente 

In [ ]:
# Initialize environment with wrapper
def make_env():
    env = RAGImprovementEnv()
    env = QueryResetWrapper(env, text_chunks, queries)  # Wrap to pass new query in reset
    return env

env = gym.vector.SyncVectorEnv([make_env])  # Use SyncVectorEnv instead of DummyVecEnv


In [40]:
from QueryResetWrapper import QueryResetWrapper

# Initialize environment with wrapper
def make_env():
    env = RAGImprovementEnv()
    env = QueryResetWrapper(env, text_chunks, queries)  # Wrap to pass new query in reset
    return env

env = gym.vector.SyncVectorEnv([make_env])  

In [ ]:
env.reset()

In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

text_chunks = ['chunk de teste 1', "query de teste chunk"]
query = "query de teste"
documents = text_chunks
num_documents = len(documents)
document_embeddings = np.array([model.encode(chunk) for chunk in documents])
embedding_dim = document_embeddings.shape[1]


In [12]:

# Update observation and action spaces dynamically
observation_space = spaces.Dict({
    "document_embeddings": spaces.Box(
        low=-np.inf, high=np.inf, shape=(num_documents, embedding_dim), dtype=np.float32
    ),
    "query_embedding": spaces.Box(
        low=-np.inf, high=np.inf, shape=(embedding_dim,), dtype=np.float32
    ),
    "similarities": spaces.Box(
        low=0.0, high=1.0, shape=(num_documents,), dtype=np.float32
    ),
    "retrieved_chunks_mask": spaces.MultiBinary(num_documents)
})
action_space = spaces.Discrete(num_documents + 1)

# Compute query embedding and similarities
query_embedding = model.encode(query)
similarities = np.array([
    util.pytorch_cos_sim(query_embedding, doc_emb).item() for doc_emb in document_embeddings
])

In [ ]:
similarities

In [1]:
# ---------------------------- Tests to solve Tensor with 4 elements on wrapper

In [1]:
from Functions.documents import load_documents
from Functions.text import clear_text, remove_stopwords, split_text_into_chunks, remove_portuguese_accents
from stable_baselines3 import PPO
from RAGImprovementEnv import RAGImprovementEnv
from QueryResetWrapper import QueryResetWrapper
import pandas as pd
import gymnasium as gym

CHUNK_SIZE = 500

# BEGIN = Setting texts and queries
dataset = pd.read_parquet("./tests/dataset/Crag Splitted/crag_dataset_0.parquet")  # Documents from dataset
texts = dataset['page_result'].copy()
queries = dataset['query'].copy()
# END = Setting texts and queries

# BEGIN = Processing documents 
cleaned_texts = [clear_text(text) for text in texts]
no_stopwords_text = [remove_stopwords(text, language='english') for text in cleaned_texts]
no_accents_portuguese = [remove_portuguese_accents(text) for text in no_stopwords_text]
lower_texts = [text.lower() for text in no_accents_portuguese]
text_chunks = [split_text_into_chunks(text, chunk_size=CHUNK_SIZE) for text in lower_texts]

# END = Processing documents 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_py

In [2]:
from QueryResetWrapper import QueryResetWrapper
from RAGImprovementEnv import RAGImprovementEnv
import gymnasium as gym

# Initialize environment with wrapper
def make_env():
    env = RAGImprovementEnv()
    env = QueryResetWrapper(env, queries, text_chunks)  # Wrap to pass new query in reset
    return env

# env = gym.vector.SyncVectorEnv([make_env])

In [3]:
env = make_env()

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:

obs = env.reset()  # Reset the environment
done = False
x = ''
while not done:
    action = env.action_space.sample()  # Choose a random action
    obs, reward, done, info = env.step(action)  # Apply the action
    print(f"Action: {action}, Reward: {reward}, Done: {done}, Info: {info}")
    break

QUERY: ['new orleans pelicans 2023 24 nba regular season stats espn skip main content skip navigation espn nfl nfl nba nba ncaam ncaam ncaaw ncaaw nhl nhl soccer soccer sports sports espn plus espn bet watch watch fantasy fantasy espn espn search new orleans pelicans follow 39 26 1st southwest division home stats schedule roster depth injuries transactions tickets external link new orleans pelicans stats 2023 24 nba teams atlanta hawks boston celtics brooklyn nets charlotte hornets chicago bulls cleveland cavaliers dallas mavericks denver nuggets detroit pistons golden state warriors houston rockets indiana pacers la clippers los angeles lakers memphis grizzlies miami heat milwaukee bucks minnesota timberwolves new york knicks oklahoma city thunder orlando magic philadelphia 76ers phoenix suns portland trail blazers sacramento kings san antonio spurs toronto raptors utah jazz washington wizards hidden 2023 24 regular season 2022 23 regular season 2021 22 postseason 2021 22 regular seas

In [5]:
x

''

In [6]:
# len(x)
x[3]

IndexError: string index out of range

In [6]:
import pickle
with open("doc_emb.pkl",'rb') as file:
    doc_emb = pickle.load(file)
    
with open("query.pkl",'rb') as file:
    query = pickle.load(file)

In [10]:
a = doc_emb[0]

In [15]:
query

array([[-0.02729641, -0.06921042, -0.02292343, ..., -0.00768987,
        -0.07832648, -0.00889415],
       [-0.01333874, -0.00622512, -0.06101113, ..., -0.00931067,
        -0.04527349, -0.01860411],
       [ 0.01568663, -0.02188834, -0.05950418, ..., -0.03068426,
        -0.04333907, -0.05077551],
       [-0.02461003, -0.04585826, -0.13070355, ..., -0.07408798,
        -0.07366109, -0.03766491]], dtype=float32)

In [14]:
a

array([-7.40856975e-02,  9.26612969e-03,  3.26922424e-02, -6.49216119e-03,
        9.04874783e-03, -6.71245903e-02,  1.75260261e-01,  3.44187766e-02,
        1.51931345e-02, -3.81899551e-02, -6.41652122e-02, -1.25884861e-02,
        2.31900103e-02,  1.07572146e-03,  6.98240995e-02,  2.72525623e-02,
       -8.67907777e-02, -5.77719230e-03, -7.16594607e-02,  3.03879604e-02,
       -1.47085741e-01,  7.23291859e-02, -1.10948114e-02, -6.86883985e-04,
       -3.48894186e-02, -6.86990656e-03, -1.03793405e-02, -2.62725372e-02,
        6.17748797e-02, -4.13938500e-02, -3.20653804e-02,  1.83474610e-03,
        6.76905438e-02,  7.44296703e-04, -1.71461254e-02, -2.48091221e-02,
       -1.40695563e-02, -6.13951124e-02,  7.21983984e-02,  6.04976043e-02,
        3.92866097e-02, -2.61628274e-02,  2.21649352e-02,  7.46596456e-02,
        9.60129946e-02, -1.18583739e-01, -8.85697640e-03, -6.62836432e-02,
       -7.93788210e-03, -5.47165871e-02,  1.73085164e-02, -1.31182116e-03,
        2.99289767e-02, -

In [11]:
from sentence_transformers import SentenceTransformer, util

util.pytorch_cos_sim(query, a)

tensor([[0.0907],
        [0.0880],
        [0.1807],
        [0.1434]])

In [9]:
a = ['new orleans pelicans 2023 24 nba regular season stats espn skip main content skip navigation espn nfl nfl nba nba ncaam ncaam ncaaw ncaaw nhl nhl soccer soccer sports sports espn plus espn bet watch watch fantasy fantasy espn espn search new orleans pelicans follow 39 26 1st southwest division home stats schedule roster depth injuries transactions tickets external link new orleans pelicans stats 2023 24 nba teams atlanta hawks boston celtics brooklyn nets charlotte hornets chicago bulls cleveland cavaliers dallas mavericks denver nuggets detroit pistons golden state warriors houston rockets indiana pacers la clippers los angeles lakers memphis grizzlies miami heat milwaukee bucks minnesota timberwolves new york knicks oklahoma city thunder orlando magic philadelphia 76ers phoenix suns portland trail blazers sacramento kings san antonio spurs toronto raptors utah jazz washington wizards hidden 2023 24 regular season 2022 23 regular season 2021 22 postseason 2021 22 regular season 2020 21 regular season 2019 20 regular season 2018 19 regular season 2017 18 postseason 2017 18 regular season 2016 17 regular season 2015 16 regular season 2014 15 postseason 2014 15 regular season 2013 14 regular season 2012 13 regular season 2011 12 regular season 2010 11 postseason 2010 11 regular season 2009 10 regular season 2008 09 postseason 2008 09 regular season 2007 08 postseason 2007 08 regular season 2006 07 regular season 2005 06 regular season 2004 05 regular season 2003 04 postseason 2003 04 regular season 2002 03 postseason 2002 03 regular season hidden splits home away boston celtics brooklyn nets new york knicks philadelphia 76ers toronto raptors chicago bulls cleveland cavaliers detroit pistons indiana pacers milwaukee bucks atlanta hawks charlotte hornets miami heat orlando magic washington wizards denver nuggets minnesota timberwolves oklahoma city thunder portland trail blazers utah jazz golden state warriors la clippers los angeles lakers phoenix suns sacramento kings dallas mavericks houston rockets memphis grizzlies new orleans pelicans san antonio spurs october november december january february march april atlantic division central division pacific division southeast division southwest division northwest division eastern conference western conference pre star post star 0 days rest 1 day rest 2 days rest 3 days rest hidden team leaders points zion williamson pf 22 3 rebounds jonas valanciunas c 9 5 assists brandon ingram sf 5 8 steals dyson daniels g 1 4 blocks jonas valanciunas c 1 0 player stats splits name zion williamson pf brandon ingram sf cj mccollum sg trey murphy iii sg jonas valanciunas c herbert jones sf jordan hawkins g matt ryan f naji marshall sf jose alvarado pg dyson daniels g larry nance jr pf izaiah brockington g jeremiah robinson earl f kira lewis jr pg dereon seabron g cody zeller c e j liddell f kaiser gates f jalen crutcher g total gp gs min pts dr reb ast stl blk pf ast 54 54 30 8 22 3 1 8 3 9 5 7 5 1 0 9 0 6 2 5 2 2 2 0 59 59 33 2 21 3 0', '7 4 4 5 1 5 8 0 8 0 6 2 6 2 3 2 2 49 49 31 2 18 3 0 6 3 7 4 3 4 3 1 0 0 7 1 6 2 0 2 8 40 10 27 2 13 9 0 6 3 5 4 2 1 9 0 8 0 5 0 5 1 3 3 8 65 65 24 8 13 0 2 6 6 9 9 5 2 2 0 4 1 0 1 4 2 8 1 6 60 60 29 3 11 1 1 1 2 5 3 6 2 5 1 3 0 9 1 2 2 9 2 1 56 10 19 3 9 0 0 5 2 0 2 5 1 2 0 3 0 1 0 7 1 2 1 7 20 1 17 7 7 4 0 3 1 7 2 0 0 8 0 3 0 1 0 9 1 5 0 9 53 1 19 0 7 2 0 9 2 7 3 6 2 0 0 7 0 2 1 1 1 4 1 9 44 0 17 3 6 9 0 4 1 8 2 2 2 0 1 1 0 3 0 8 1 4 2 6 52 15 21 9 5 5 1 0 2 7 3 7 2 6 1 4 0 4 1 0 1 6 2 6 47 0 18 7 5 3 1 4 3 6 5 1 1 6 1 0 0 4 0 9 1 5 1 9 1 0 3 0 4 0 1 0 1 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 28 1 9 5 3 4 0 6 1 5 2 1 0 5 0 3 0 1 0 4 0 6 1 1 15 0 9 6 2 9 0 3 0 7 0 9 1 2 0 3 0 1 0 5 0 5 2 3 6 0 9 2 2 3 0 2 1 0 1 2 0 8 0 3 0 2 0 0 0 7 inf 39 0 7 7 1 9 1 2 1 6 2 7 0 9 0 2 0 1 0 4 1 1 2 4 8 0 2 9 0 5 0 0 0 6 0 6 0 1 0 3 0 3 0 3 0 3 0 5 1 0 7 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 65 116 1 10 8 33 9 44 7 27 0 8 2 5 0 12 7 18 8 2 1 shooting stats splits name zion williamson pf brandon ingram sf cj mccollum sg trey murphy iii sg jonas valanciunas c herbert jones sf jordan hawkins g matt ryan f naji marshall', 'sf jose alvarado pg dyson daniels g larry nance jr pf izaiah brockington g jeremiah robinson earl f kira lewis jr pg dereon seabron g cody zeller c e j liddell f kaiser gates f jalen crutcher g total fgm fga fg 3pm 3pa 3p ftm fta ft 2pm 2pa 2p sc eff sh eff 8 8 15 2 57 8 0 1 0 3 35 7 4 7 6 7 69 7 8 7 14 9 58 2 1 471 0 58 7 9 16 1 49 3 1 4 3 8 36 2 4 1 5 1 80 3 6 6 12 3 53 4 1 324 0 54 6 7 14 7 45 3 3 2 7 9 41 2 1 7 2 2 79 4 3 4 6 8 50 2 1 246 0 56 4 7 10 5 44 2 2 9 7 6 37 4 1 8 2 3 76 9 1 8 2 9 62 1 1 321 0 58 5 2 9 2 56 2 0 6 1 6 35 6 2 1 2 6 80 2 4 6 7 7 60 3 1 413 0 59 4 0 7 7 51 4 1 5 3 5 43 5 1 7 2 0 85 0 2 4 4 2 57 9 1 447 0 61 3 0 7 7 38 9 1 9 5 0 36 9 1 2 1 4 84 4 1 1 2 7 42 7 1 169 0 51 2 4 5 2 45 2 2 0 4 3 47 1 0 7 0 7 92 9 0 4 1 0 36 8 1 413 0 64 2 6 5 6 45 9 0 9 2 3 38 5 1 2 1 5 79 3 1 7 3 3 51 1 1 297 0 54 2 5 6 0 41 4 1 3 3 5 36 2 0 7 1 0 69 0 1 3 2 6 48 2 1 143 0 52 2 2 4 9 43 8 0 6 2 1 29 1 0 6 1 0 64 0 1 5 2 8 54 8 1 125 0 50 2 1 3 6 57 9 0 4 1 0 43 8 0 7 0 9 72 7 1 7 2 6 63 4 1 468 0 64 2 0 5 0 40 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 4 0 50 0 0 800 0 40 1 3 2 5 49 3 0 4 1 1 40 0 0 4 0 5 80 0 0 8 1 5 56 1 1 324 0 58 1 1 3 5 30 8 0 1 1 3 10 0 0 7 0 7 90 9 0 9 2 1 43 8 0 846 0 33 0 7 2 2 30 8 0 2 0 3 50 0 0 8 1 0 83 3 0 5 1 8 27 3 1', '077 0 35 0 6 1 5 41 7 0 0 0 1 33 3 0 6 0 9 61 1 0 6 1 5 42 1 1 217 0 43 0 1 0 8 16 7 0 0 0 6 0 0 0 3 0 3 100 0 0 1 0 1 100 0 0 667 0 17 0 0 4 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 000 0 00 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 000 0 00 42 8 87 8 48 8 12 3 32 3 38 1 18 1 23 6 76 7 30 5 55 5 55 0 1 321 0 56 midseason trades kira lewis jr 1 gp tor statistics updated nightly glossary gp games played gs games started min minutes per game pts points per game offensive rebounds per game dr defensive rebounds per game reb rebounds per game ast assists per game stl steals per game blk blocks per game turnovers per game pf fouls per game ast assist turnover ratio fgm average field goals made fga average field goals attempted fg field goal percentage 3pm average 3 point field goals made 3pa average 3 point field goals attempted 3p 3 point field goal percentage ftm average free throws made fta average free throws attempted ft free throw percentage 2pm 2 point field goals made per game 2pa 2 point field goals attempted per game 2p 2 point field goal percentage sc eff scoring efficiency sh eff shooting efficiency terms use privacy policy us state privacy rights children online privacy policy interest based ads nielsen measurement sell share personal information contact us disney ad sales site work espn corrections espn bet operated penn entertainment inc subsidiaries penn espn bet available states penn licensed offer sports wagering must 21 wager someone know gambling problem wants help call 1 800 gambler copyright 2024 espn enterprises inc rights reserved']


In [10]:
a

['new orleans pelicans 2023 24 nba regular season stats espn skip main content skip navigation espn nfl nfl nba nba ncaam ncaam ncaaw ncaaw nhl nhl soccer soccer sports sports espn plus espn bet watch watch fantasy fantasy espn espn search new orleans pelicans follow 39 26 1st southwest division home stats schedule roster depth injuries transactions tickets external link new orleans pelicans stats 2023 24 nba teams atlanta hawks boston celtics brooklyn nets charlotte hornets chicago bulls cleveland cavaliers dallas mavericks denver nuggets detroit pistons golden state warriors houston rockets indiana pacers la clippers los angeles lakers memphis grizzlies miami heat milwaukee bucks minnesota timberwolves new york knicks oklahoma city thunder orlando magic philadelphia 76ers phoenix suns portland trail blazers sacramento kings san antonio spurs toronto raptors utah jazz washington wizards hidden 2023 24 regular season 2022 23 regular season 2021 22 postseason 2021 22 regular season 2020

In [11]:
dataset = pd.read_parquet("./tests/dataset/Crag Splitted/crag_dataset_0.parquet")  # Documents from dataset


In [21]:
dataset['query'].values[100]

"what percentage of gross for pixar's 2020 movies came from outside the us and canada?"